In [1]:
from flask import Flask, render_template, request
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

In [3]:
app = Flask(__name__)
model = load_model('/content/drive/MyDrive/malaria/autoencoder_model')

In [4]:
def check_anomaly(img_path):
    density_threshold = 2700 # Set this value based on the above exercise
    reconstruction_error_threshold = 0.0089 # Set this value based on the above exercise

    # Load the image
    img = Image.open(img_path)
    img = np.array(img.resize((128, 128), Image.ANTIALIAS))
    plt.imshow(img)
    plt.show()

    # Preprocess the image
    img = img / 255.
    img = img[np.newaxis, :, :, :]

    # Encode the image
    encoded_img = encoder_model.predict([[img]])
    encoded_img = [np.reshape(img, out_vector_shape) for img in encoded_img]

    # Calculate density and reconstruction error
    density = kde.score_samples(encoded_img)[0]
    reconstruction = model.predict([[img]])
    reconstruction_error = model.evaluate([reconstruction], [[img]], batch_size=1)[0]

    # Determine if the image is an anomaly
    if density < density_threshold or reconstruction_error > reconstruction_error_threshold:
        print("The image is an anomaly")
    else:
        print("The image is NOT an anomaly")

In [5]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if request.method == 'POST':
        file = request.files['file']
        if file:
            img_path = '/tmp/uploaded_image.jpg'
            file.save(img_path)
            result = check_anomaly_random(img_path)
            return render_template('index.html', result=result)
    return "No file uploaded"

In [6]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
